In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import h5py
import matplotlib.pyplot as plt
from dynpar import evaluate

import warnings
warnings.filterwarnings("ignore", category=h5py.h5py_warnings.H5pyDeprecationWarning)

In [ ]:
list_subject = ['sub-01','sub-02', 'sub-03', 'sub-04', 'sub-05', 'sub-06']

list_clusters = [20, 50]
list_states = [150, 300]
list_fwhm = [8]

In [ ]:
main_path = 'path to your hdf5 files'
output_folder = os.path.join(main_path, 'stats')

In [ ]:
all_reproducibility_df = pd.DataFrame()

In [ ]:
for subject in list_subject:
    
    for fwhm in list_fwhm:
        
        for cluster in list_clusters:

            for states in list_states:
                
                print('subject', subject)
                print('fwhm', fwhm)
                print('cluster', cluster)
                print('states', states)
    
                try:
            
                    out_filename_within = os.path.join(main_path, 'within-subject-reproducibility', 'within-'+subject+'-reproducibility-fwhm-'+str(fwhm)+'.h5')

                    h5_data = h5py.File(out_filename_within, 'a')

                    reproducibility_scores = h5_data[str(subject)][str(fwhm)][str(cluster)][str(states)]['States-reproducibility'][:]

                    print(reproducibility_scores)

                    w_sub_rep = reproducibility_scores[reproducibility_scores != 0]

                    data = {'Pearson correlation' : w_sub_rep,  
                            'Subject': np.repeat(subject, len(w_sub_rep)),
                            'Reproducibility level': 'Within-subjects', 
                            'Fwhm-clusters-states': np.repeat('F:'+str(fwhm)+'-C:'+str(cluster)+'-S:'+str(states), len(w_sub_rep)),
                            }

                    all_reproducibility_df = pd.concat([all_reproducibility_df, pd.DataFrame(data=data)])    
                except:
                    continue

In [ ]:
for idx1 in range(len(list_subject)):
    
    subject1 = list_subject[idx1]
    
    for idx2 in range(idx1+1, len(list_subject)):
    
        for fwhm in list_fwhm:

            for cluster in list_clusters:

                for states in list_states:

                    try:
                        subject2 = list_subject[idx2]
                 
                        out_filename_between = os.path.join(main_path, 'between-subjects-reproducibility', 'between-subjects-'+subject1+'-'+subject2+'-reproducibility-fwhm'+str(fwhm)+'.h5')

                        print(out_filename_between)
                        
                        h5_data = h5py.File(out_filename_between, 'a')
                        
                        print(list(h5_data.keys()))

                        reproducibility_scores = h5_data[subject1+'-'+subject2][str(fwhm)][str(cluster)][str(states)]['States-reproducibility'][:]

                        w_sub_rep = reproducibility_scores[reproducibility_scores != 0]

                        data = {'Pearson correlation' : w_sub_rep,  
                                'Subjects': np.repeat(subject1+'-'+subject2, len(w_sub_rep)),
                                'Reproducibility level': 'Between-subjects',
                                'Fwhm-clusters-states': np.repeat('F:'+str(fwhm)+'-C:'+str(cluster)+'-S:'+str(states), len(w_sub_rep)),
                                }

                        all_reproducibility_df = pd.concat([all_reproducibility_df, pd.DataFrame(data=data)])    
                    except:
                        continue

In [ ]:
all_reproducibility_df

In [ ]:
ymin = -0.2
ymax = 1.1

figure, ax = plt.subplots(1, 1,figsize=(10,7))

plt.rcParams.update({'font.size': 13})


ax.set_ylim(ymin, ymax) 
ax.set_ylim(ymin, ymax) 


In [ ]:
tab10 = "tab10"

m = plt.cm.get_cmap(tab10)


sns.swarmplot(x='Reproducibility level', y='Pearson correlation', hue='Fwhm-clusters-states',
                   dodge=True, data=all_reproducibility_df, alpha=0.5, ax=ax)

sns.boxplot(x='Reproducibility level', y='Pearson correlation', data=all_reproducibility_df, hue='Fwhm-clusters-states',
                dodge=True, showfliers = False, boxprops=dict(alpha=0.3),
                 showmeans=True, ax=ax)

In [ ]:
figure

In [ ]:

filename_figure = os.path.join(main_path, 'plots','spatial-reproducibility.png')
figure.savefig(filename_figure)
plt.show()